In [1]:
# ============================================
# 0) Preparación e instalacion de paquetes
# ============================================
import sys

# 1) Limpiar posibles instalaciones incompatibles previas
!pip -q uninstall -y numpy pandas >/dev/null

# 2) Instalar versiones compatibles y el dependencias
!{sys.executable} -m pip -q install --no-cache-dir --upgrade \
  "numpy==2.0.2" \
  "pandas==2.2.2" \
  "matplotlib>=3.9,<3.10" \
  "seaborn==0.13.2" \
  "plotly==5.23.0" \
  "openpyxl==3.1.5" \
  "sodapy==2.2.0" \
  "python-dotenv==1.0.1"

# 3) Mostrar versiones cargadas
import numpy as np, pandas as pd, matplotlib, seaborn, plotly, openpyxl
print("Versiones cargadas →",
      "Python:", sys.version.split()[0],
      "| NumPy:", np.__version__,
      "| pandas:", pd.__version__,
      "| matplotlib:", matplotlib.__version__,
      "| seaborn:", seaborn.__version__,
      "| plotly:", plotly.__version__)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 135.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 58.1 MB/s eta 0:00:00
Versiones cargadas → Python: 3.12.11 | NumPy: 2.0.2 | pandas: 2.2.2 | matplotlib: 3.10.0 | seaborn: 0.13.2 | plotly: 5.23.0


In [2]:
# ============================================
# 1) Definiciones Generales y parametros
# ============================================

# Imports mínimos necesarios para este bloque
import os, sys, io, warnings
import pandas as pd
import numpy as np
from IPython.display import display

warnings.filterwarnings("ignore")

#Parámetros
PROJECT_NAME = "acceso_vivienda_col_4ciudades_2020_2024"

# User-Agent personalizado (para usar en llamadas HTTP)
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36"

# Token público de datos.gov.co (opcional). Si tienes un APP token, guárdalo como variable de entorno en Colab:
APP_TOKEN = os.getenv("DATOSGOV_APP_TOKEN", "")

# Ciudades objetivo segun códigos DANE
CITY_MAP = {
    "Bogotá":      {"codigo_municipio": "11001", "alias":"Bogotá D.C."},
    "Medellín":    {"codigo_municipio": "05001", "alias":"Medellín"},
    "Cali":        {"codigo_municipio": "76001", "alias":"Santiago de Cali"},
    "Barranquilla":{"codigo_municipio": "08001", "alias":"Barranquilla"}
}

# Temporalidad
YEAR_START, YEAR_END = 2020, 2024

# Supuestos para cálculos financieros
LTV = 0.80             # préstamo / valor vivienda
DOWNPAYMENT_PCT = 0.20 # cuota inicial
PLAZO_YEARS = 20
AHORRO_TASA = 0.10
INGRESOS_ESCENARIOS_SMMLV = [2, 3, 4]

#Carpeta del proyecto y cambio de directorio
if not os.path.exists(PROJECT_NAME):
    os.makedirs(PROJECT_NAME, exist_ok=True)
os.chdir(PROJECT_NAME)

# Validaciones de generacion
print("Directorio de trabajo:", os.getcwd())

cfg_df = pd.DataFrame({
    "clave": ["USER_AGENT","PROJECT_NAME","APP_TOKEN_definido","YEAR_START","YEAR_END","LTV","DOWNPAYMENT_PCT","PLAZO_YEARS","AHORRO_TASA","INGRESOS_SMMLV"],
    "valor": [
        USER_AGENT[:60] + ("..." if len(USER_AGENT)>60 else ""),
        PROJECT_NAME,
        bool(APP_TOKEN),
        YEAR_START,
        YEAR_END,
        LTV,
        DOWNPAYMENT_PCT,
        PLAZO_YEARS,
        AHORRO_TASA,
        str(INGRESOS_ESCENARIOS_SMMLV)
    ]
})
print("Configuración general cargada:")
display(cfg_df)

city_df = (pd.DataFrame(CITY_MAP).T
             .reset_index()
             .rename(columns={"index":"ciudad"}))
print("Ciudades objetivo (códigos DANE):")
display(city_df)


Directorio de trabajo: /content/acceso_vivienda_col_4ciudades_2020_2024
Configuración general cargada:


,clave,valor
0,USER_AGENT,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
1,PROJECT_NAME,acceso_vivienda_col_4ciudades_2020_2024
2,APP_TOKEN_definido,False
3,YEAR_START,2020
4,YEAR_END,2024
5,LTV,0.8
6,DOWNPAYMENT_PCT,0.2
7,PLAZO_YEARS,20
8,AHORRO_TASA,0.1
9,INGRESOS_SMMLV,"[2, 3, 4]"


Ciudades objetivo (códigos DANE):


,ciudad,codigo_municipio,alias
0,Bogotá,11001,Bogotá D.C.
1,Medellín,05001,Medellín
2,Cali,76001,Santiago de Cali
3,Barranquilla,08001,Barranquilla


In [3]:
#Credenciales para extarr informacion de socrata, datos abiertos
import os
os.environ["DATOSGOV_APP_TOKEN"] = "wqtP1g7fgTKTtIwD09w6auAns"
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36"


In [4]:
# ============================================================
# 2) Extraccion API: SFC Vivienda (kett-344s), nacional mensual
# solo estaba la informacion del 2022 al 2024
# No se ejecuto por errores de conexion, y demoras en ejecucion
# ============================================================
import os, requests, time
import pandas as pd
import numpy as np

DOMAIN   = "www.datos.gov.co"
DATASET  = "kett-344s"   # Tasas de interés de vivienda (SFC)
APP_TOKEN = os.getenv("DATOSGOV_APP_TOKEN", "")
USER_AGENT = globals().get("USER_AGENT", "Mozilla/5.0 (X11; Linux x86_64)")
HEADERS  = {"User-Agent": USER_AGENT}
if APP_TOKEN:
    HEADERS["X-App-Token"] = APP_TOKEN

YEAR_START, YEAR_END = 2022, 2024
START = f"{YEAR_START}-01-01T00:00:00.000"
END   = f"{YEAR_END}-12-31T23:59:59.999"

CITY_MAP = {
    "Bogotá": {"codigo":"11001"},
    "Medellín": {"codigo":"05001"},
    "Cali": {"codigo":"76001"},
    "Barranquilla": {"codigo":"08001"},
}

def _req(params=None, query=None, timeout=30, retries=3):
    url = f"https://{DOMAIN}/resource/{DATASET}.json"
    for a in range(retries):
        try:
            if query is not None:
                r = requests.get(url, headers=HEADERS, params={"$query": query}, timeout=timeout)
            else:
                r = requests.get(url, headers=HEADERS, params=params or {}, timeout=timeout)
            if r.status_code == 200:
                return r.json()
            else:
                print(f"HTTP {r.status_code}. Respuesta del servidor:\n{r.text[:400]}\n")
                wait = (2**a) + np.random.uniform(0,1)
                print(f"Reintento en {wait:.1f}s…")
                time.sleep(wait)
        except Exception as e:
            wait = (2**a) + np.random.uniform(0,1)
            print(f"Excepción {type(e).__name__}: {e}. Reintento en {wait:.1f}s…")
            time.sleep(wait)
    raise RuntimeError("No fue posible obtener respuesta de la API tras varios reintentos.")

# 1)Primera aproximacion, ejecutando query
q_nat = f"""
SELECT
  date_trunc_ym(fecha_corte) AS mes,
  count(tasa_efectiva_promedio_ponderada) AS n_obs,
  avg(tasa_efectiva_promedio_ponderada) AS tasa_ea_prom_pct
WHERE
  fecha_corte between '{START}' and '{END}'
  AND upper(tipo_de_tasa) like '%EA%'
GROUP BY mes
ORDER BY mes
"""
print("Consultando (server) nacional mensual (EA)…")
js = _req(query=q_nat)
df_nat = pd.DataFrame(js)

# 2) Segunda aproximacion
if df_nat.empty:
    print("Sin filas EA a nivel nacional. Fallback: semanal (EA+NOM) y agregamos en cliente…")
    params_fb = {
        "$where": f"fecha_corte between '{START}' and '{END}'",
        "$select": "fecha_corte,tipo_de_tasa,tasa_efectiva_promedio_ponderada",
        "$order": "fecha_corte",
        "$limit": 500000
    }
    js2 = _req(params=params_fb)
    raw = pd.DataFrame(js2)
    if raw.empty:
        raise SystemExit("No se obtuvieron datos de 'kett-344s'. Verifica fechas/token.")
    raw["fecha_corte"] = pd.to_datetime(raw["fecha_corte"], errors="coerce")
    raw = raw.dropna(subset=["fecha_corte"]).copy()
    raw["mes"] = raw["fecha_corte"].dt.to_period("M").dt.to_timestamp()
    raw["tasa"] = pd.to_numeric(raw["tasa_efectiva_promedio_ponderada"], errors="coerce")
    is_nom = raw["tipo_de_tasa"].str.upper().str.contains("NOM", na=False)
    raw["ea"] = np.where(is_nom, (1 + raw["tasa"]/100.0)**12 - 1, raw["tasa"]/100.0)
    sfc_nat_month = (raw.groupby("mes", as_index=False)
                        .agg(n_obs=("ea","size"), tasa_media_ea=("ea","mean")))
else:
    df_nat["mes"] = pd.to_datetime(df_nat["mes"], errors="coerce")
    df_nat["n_obs"] = pd.to_numeric(df_nat["n_obs"], errors="coerce")
    df_nat["tasa_media_ea"] = pd.to_numeric(df_nat["tasa_ea_prom_pct"], errors="coerce")/100.0
    sfc_nat_month = df_nat[["mes","n_obs","tasa_media_ea"]].copy()

# 3) Tasa mensual y salidas
sfc_nat_month["tasa_mensual"] = (1 + sfc_nat_month["tasa_media_ea"])**(1/12) - 1
sfc_nat_month["tipo_tasa"] = "EA"

# Replica a las 4 ciudades
rows = []
for _, r in sfc_nat_month.iterrows():
    for city in CITY_MAP.keys():
        rows.append({
            "ciudad": city,
            "mes": r["mes"],
            "tasa_media_ea": r["tasa_media_ea"],
            "tipo_tasa": "EA",
            "n_obs": r["n_obs"],
            "tasa_mensual": r["tasa_mensual"]
        })
sfc_month = pd.DataFrame(rows)

def safe_save_csv(df, path):
    tmp = path + ".tmp"
    df.to_csv(tmp, index=False)
    os.replace(tmp, path)

safe_save_csv(sfc_nat_month, "sfc_tasas_mensuales_nacional.csv")
safe_save_csv(sfc_month,      "sfc_tasas_mensuales_ciudad.csv")

print("Listo: sfc_tasas_mensuales_nacional.csv y sfc_tasas_mensuales_ciudad.csv")
print("Meses por año (post filtro):")
print(sfc_nat_month.assign(year=sfc_nat_month["mes"].dt.year).groupby("year")["mes"].count())


Consultando (server) nacional mensual (EA)…
HTTP 400. Respuesta del servidor:
{"message":"Query coordinator error: query.soql.no-such-column; No such column: fecha_corte; position: Map(row -> 1, column -> 32, line -> \"SELECT |> SELECT date_trunc_ym(`fecha_corte`) AS `mes`, count(`tasa_efectiva_promedio_ponderada`) AS `n_obs`, avg(`tasa_efectiva_promedio_ponderada`) AS `tasa_ea_prom_pct` WHERE `fecha_corte` BETWEEN \\\"2022-01-01T00:00:00.000\\\" AND \\\"2024-12-31T23:59:59

Reintento en 1.5s…
HTTP 400. Respuesta del servidor:
{"message":"Query coordinator error: query.soql.no-such-column; No such column: fecha_corte; position: Map(row -> 1, column -> 32, line -> \"SELECT |> SELECT date_trunc_ym(`fecha_corte`) AS `mes`, count(`tasa_efectiva_promedio_ponderada`) AS `n_obs`, avg(`tasa_efectiva_promedio_ponderada`) AS `tasa_ea_prom_pct` WHERE `fecha_corte` BETWEEN \\\"2022-01-01T00:00:00.000\\\" AND \\\"2024-12-31T23:59:59

Reintento en 2.7s…
HTTP 400. Respuesta del servidor:
{"message":"

RuntimeError: No fue posible obtener respuesta de la API tras varios reintentos.

In [6]:
# ============================================================
# 2B) Extraccion tasas Banco de la Republica
# Se usan tasas EA mensuales desde un CSV local (BanRep),
# con limpieza y agregación nacional, replicada a 4 ciudades.
# Periodo: 2020-2024
# ============================================================

import os
import pandas as pd
import numpy as np
import unicodedata
from datetime import datetime
from pathlib import Path
import re

BANREP_CSV  = "/content/acceso_vivienda_col_4ciudades_2020_2024/banrep_vivienda_pesos_mensual.csv"
YEAR_START, YEAR_END = 2020, 2024
CITY_MAP = {
    "Bogotá":       {"codigo":"11001"},
    "Medellín":     {"codigo":"05001"},
    "Cali":         {"codigo":"76001"},
    "Barranquilla": {"codigo":"08001"},
}

# -------- Helpers de IO --------
def _read_any_csv(src, **kwargs):
    """
    Lee CSV (o Excel) desde ruta local o URL con heurísticas de separador/encoding.
    """
    p = str(src)
    ext = Path(p).suffix.lower()

    if ext in ('.xlsx', '.xls'):
        return pd.read_excel(p, **{k: v for k, v in kwargs.items() if k != 'engine'})

    # 1) Inferir separador y usar UTF-8
    try:
        return pd.read_csv(p, sep=None, engine='python', encoding='utf-8', **kwargs)
    except Exception:
        pass

    # 2) Codificaciones comunes en Colombia
    for enc in ('latin-1', 'cp1252'):
        try:
            return pd.read_csv(p, sep=None, engine='python', encoding=enc, **kwargs)
        except Exception:
            continue

    # 3) Asumir ';' como separador (muy común)
    try:
        return pd.read_csv(p, sep=';', engine='python', encoding='utf-8', **kwargs)
    except Exception:
        return pd.read_csv(p, sep=';', engine='python', encoding='latin-1', **kwargs)

# -------- Limpieza y utilidades --------
def _norm_name(s):
    """Normaliza nombre de columna: minúsculas, sin acentos, sin espacios dobles."""
    s = str(s).strip()
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    s = s.lower()
    s = s.replace("\ufeff","").replace("\t"," ")
    while "  " in s:
        s = s.replace("  "," ")
    return s

def _find_col(cols_norm, patterns):
    """Busca la primera columna cuyo nombre normalizado contenga alguno de los patrones (regex)."""
    for pat in patterns:
        rx = re.compile(pat, re.I)
        for c_norm, c_original in cols_norm.items():
            if rx.search(c_norm):
                return c_original
    return None

def _to_num(x):
    """Convierte string con coma/punto a número."""
    if isinstance(x, str):
        x = x.strip().replace(" ", "")
        # "1.234,56" -> "1234.56"
        if x.count(",") == 1 and x.count(".") >= 1 and x.rfind(",") > x.rfind("."):
            x = x.replace(".", "").replace(",", ".")
        else:
            x = x.replace(",", ".")
    return pd.to_numeric(x, errors="coerce")

# -------- 1) Leer CSV --------
if not Path(BANREP_CSV).exists():
    raise FileNotFoundError(f"No encuentro el archivo en: {BANREP_CSV}.\n"
                            "Verifica la ruta o monta Drive si aplica:\n"
                            "from google.colab import drive\n"
                            "drive.mount('/content/drive')")

raw = _read_any_csv(BANREP_CSV)
cols_norm = {_norm_name(c): c for c in raw.columns}

print("Columnas detectadas en el CSV de BanRep:")
for k, v in cols_norm.items():
    print(f"  - {v}  →  {k}")

# -------- 2) Detectar columnas de fecha y valor --------
date_col  = _find_col(cols_norm, [r"\bfecha\b", r"period", r"mes.*anio", r"anio.*mes", r"\bmes\b"])
year_col  = _find_col(cols_norm, [r"\ba[nñ]o\b", r"\banio\b", r"\byear\b"])
month_col = _find_col(cols_norm, [r"\bmes\b", r"\bmonth\b"])

# Patrón más estricto para la tasa EA (evita confundir 'ea' con 'media')
value_col = _find_col(
    cols_norm,
    [r"\bdato\b", r"\bvalor\b", r"\btasa\b", r"\b(e\.?a\.?|efectiva.*anual|tasa.*anual)\b", r"\bporcentaje\b"]
)
if value_col is None:
    raise SystemExit("No pude detectar la columna del valor (EA %). Revisa los nombres de columnas.")

# -------- 3) Construir columna 'mes' --------
df = raw.copy()

def _build_mes_from_columns(df, date_col, year_col, month_col):
    mes = None
    if date_col is not None:
        # Intentar parsear 'Fecha' o 'Periodo' con diferentes formatos
        try:
            mes = pd.to_datetime(df[date_col], errors="coerce", dayfirst=True)
        except Exception:
            mes = pd.to_datetime(df[date_col].astype(str).str[:10], errors="coerce")
        # si trae día, lo truncamos al primer día del mes
        if mes.notna().any():
            mes = mes.dt.to_period("M").dt.to_timestamp()
    elif year_col is not None and month_col is not None:
        yy = pd.to_numeric(df[year_col], errors="coerce")
        mm = pd.to_numeric(df[month_col], errors="coerce")
        # relleno básico por si mes está como nombre
        if mm.isna().any():
            meses = {m.lower(): i for i, m in enumerate(
                ["", "enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","diciembre"]
            )}
            mm = df[month_col].astype(str).str.lower().map(meses)
        mes = pd.to_datetime(
            dict(year=yy.fillna(2000).astype(int),
                 month=mm.fillna(1).astype(int),
                 day=1),
            errors="coerce"
        )
    return mes

mes = _build_mes_from_columns(df, date_col, year_col, month_col)
if mes is None or mes.isna().all():
    # si hay 'periodo' tipo 'YYYY-MM' en texto
    if date_col is not None:
        mes = pd.to_datetime(df[date_col].astype(str).str.replace("/", "-"), errors="coerce")
        mes = mes.dt.to_period("M").dt.to_timestamp()
    if mes is None or mes.isna().all():
        raise SystemExit("No pude construir la columna 'mes' a partir de las columnas de fecha/periodo.")

df["mes"] = mes
df = df.dropna(subset=["mes"]).copy()

# -------- 4) Convertir EA% a decimal y tasa mensual --------
df["ea_pct"] = df[value_col].apply(_to_num)
df = df.dropna(subset=["ea_pct"]).copy()

# Filtrar rango temporal
df = df[(df["mes"].dt.year >= YEAR_START) & (df["mes"].dt.year <= YEAR_END)].copy()
if df.empty:
    raise SystemExit(f"No hay datos en el rango {YEAR_START}–{YEAR_END}.")

df["tasa_media_ea"] = df["ea_pct"] / 100.0
df["tasa_mensual"]  = (1.0 + df["tasa_media_ea"])**(1/12) - 1
df["n_obs"]         = 1
df["tipo_tasa"]     = "EA"

# -------- 5) Agregación por mes (nacional) --------
sfc_nat_month = (
    df.groupby("mes", as_index=False)
      .agg(tasa_media_ea=("tasa_media_ea", "mean"),
           n_obs=("n_obs", "size"))
)
sfc_nat_month["tasa_mensual"] = (1 + sfc_nat_month["tasa_media_ea"])**(1/12) - 1
sfc_nat_month["tipo_tasa"]    = "EA"

# -------- 6) Replicar nacional a 4 ciudades --------
rows = []
for _, r in sfc_nat_month.iterrows():
    for city in CITY_MAP.keys():
        rows.append({
            "ciudad": city,
            "mes": r["mes"],
            "tasa_media_ea": r["tasa_media_ea"],
            "tipo_tasa": "EA",
            "n_obs": r["n_obs"],
            "tasa_mensual": r["tasa_mensual"]
        })
sfc_month = pd.DataFrame(rows)

# -------- 7) Guardar salidas --------
def safe_save_csv(df, path):
    tmp = path + ".tmp"
    df.to_csv(tmp, index=False)
    os.replace(tmp, path)

safe_save_csv(sfc_nat_month, "sfc_tasas_mensuales_nacional.csv")
safe_save_csv(sfc_month,      "sfc_tasas_mensuales_ciudad.csv")

print("Guardados:")
print("- sfc_tasas_mensuales_nacional.csv")
print("- sfc_tasas_mensuales_ciudad.csv")

print("\nMeses por año (nacional):")
print(
    sfc_nat_month.assign(year=sfc_nat_month["mes"].dt.year)
                 .groupby("year")["mes"].count()
)

print("\nMapeo usado:")
print(f"  • Columna de fecha/periodo usada: {date_col or (year_col + ' + ' + month_col if year_col and month_col else 'detectada por patrón')}")
print(f"  • Columna de valor usada: {value_col}")


Columnas detectadas en el CSV de BanRep:
  - fecha  →  fecha
  - tasa  →  tasa
Guardados:
- sfc_tasas_mensuales_nacional.csv
- sfc_tasas_mensuales_ciudad.csv

Meses por año (nacional):
year
2020    12
2021    12
2022    12
2023    12
2024    12
Name: mes, dtype: int64

Mapeo usado:
  • Columna de fecha/periodo usada: fecha
  • Columna de valor usada: tasa


In [8]:
# ============================================
# 2B.1) Visualización informacion extraida de BanRep
# ============================================

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

nat = pd.read_csv("sfc_tasas_mensuales_nacional.csv", parse_dates=["mes"])
for c in ["tasa_media_ea","tasa_mensual","n_obs"]:
    if c in nat.columns:
        nat[c] = pd.to_numeric(nat[c], errors="coerce")

nat = nat.sort_values("mes").copy()
nat["mm3"] = nat["tasa_mensual"].rolling(3, min_periods=1).mean()
years = nat["mes"].dt.year.unique()
y0, y1 = int(years.min()), int(years.max())

# 1) Línea mensual con media móvil 3M + anotaciones de extremos
i_max = nat["tasa_mensual"].idxmax()
i_min = nat["tasa_mensual"].idxmin()

fig1 = go.Figure()
fig1.add_trace(go.Scatter(
    x=nat["mes"], y=nat["tasa_mensual"],
    mode="lines+markers", name="Tasa mensual",
    line=dict(color="#1f77b4")
))
fig1.add_trace(go.Scatter(
    x=nat["mes"], y=nat["mm3"],
    mode="lines", name="Media móvil 3M",
    line=dict(color="#ff7f0e", dash="dash")
))

if not np.isnan(i_max):
    fig1.add_trace(go.Scatter(
        x=[nat.loc[i_max, "mes"]], y=[nat.loc[i_max, "tasa_mensual"]],
        mode="markers+text", text=["Máximo"], textposition="top center",
        name="Máximo", marker=dict(color="#d62728", size=10)
    ))
if not np.isnan(i_min):
    fig1.add_trace(go.Scatter(
        x=[nat.loc[i_min, "mes"]], y=[nat.loc[i_min, "tasa_mensual"]],
        mode="markers+text", text=["Mínimo"], textposition="bottom center",
        name="Mínimo", marker=dict(color="#2ca02c", size=10)
    ))

fig1.update_layout(
    title=f"Tasa hipotecaria promedio — nacional (mensual) {y0}–{y1}",
    xaxis_title="Mes", yaxis_title="Tasa mensual",
    template="plotly_white", hovermode="x unified"
)
fig1.update_yaxes(tickformat=".2%")
fig1.update_xaxes(rangeslider_visible=True)
fig1.show()

# 2) Barras anuales (EA) con etiquetas y cobertura (n_meses)
nat_y = (nat.assign(year=nat["mes"].dt.year)
            .groupby("year", as_index=False)
            .agg(tasa_ea_prom=("tasa_media_ea","mean"),
                 n_meses=("tasa_mensual","size")))

fig2 = px.bar(
    nat_y, x="year", y="tasa_ea_prom",
    text=nat_y["tasa_ea_prom"].map(lambda v: f"{v:.2%}"),
    labels={"year":"Año", "tasa_ea_prom":"Tasa EA promedio (nacional)"},
    title="Tasa EA promedio por año — Nacional"
)
fig2.update_traces(textposition="outside", marker_color="#4e79a7")
fig2.update_layout(
    template="plotly_white", xaxis=dict(dtick=1), yaxis_tickformat=".2%",
    annotations=[
        dict(x=int(r.year), y=float(r.tasa_ea_prom), xanchor="center", yanchor="bottom",
             text=f"<b>n={int(r.n_meses)}</b>", showarrow=False, yshift=18, font=dict(color="#333"))
        for r in nat_y.itertuples(index=False)
    ]
)
fig2.show()


out_dir = Path.cwd() / "figs_BanRep"
out_dir.mkdir(parents=True, exist_ok=True)

# (2) Nombres de archivos (usando y0, y1 que ya calculaste)
fname_line_html = out_dir / f"banrep_tasa_mensual_{y0}_{y1}.html"
fname_bar_html  = out_dir / f"banrep_tasa_ea_anual_{y0}_{y1}.html"

# (3) Guardar HTML
fig1.write_html(str(fname_line_html), include_plotlyjs="cdn", full_html=True)
fig2.write_html(str(fname_bar_html),  include_plotlyjs="cdn", full_html=True)




In [9]:
# ============================================
# 3) Lectura de IPVN (DANE) por medio de archivo local excel (no habia API)
# ============================================
import os, io, re
import pandas as pd
from IPython.display import display

# 1) Ruta del archivo local
IPVN_LOCAL_FILE = "/content/acceso_vivienda_col_4ciudades_2020_2024/anex-IPVN-VarAnualAreaUrbaMetro-IItrim2025.xlsx"  # <- tu archivo

# 2) Parámetros
YEAR_START, YEAR_END = 2020, 2024
CITIES = ["Bogotá", "Medellín", "Cali", "Barranquilla"]

# Método de "annualización":
AGG_METHOD = "Q4"   # "Q4": toma el valor del IV trimestre

# Utilidades
def try_read_target_sheet(xls):
    """
    Busca en las hojas la tabla que tenga columnas: Año, Trimestre y al menos una ciudad.
    Prueba varios encabezados (header=0..4) por si el título 'Anual' ocupa la primera fila.
    """
    for sh in xls.sheet_names:
        for hdr in range(0, 5):
            try:
                df = pd.read_excel(xls, sh, header=hdr, engine="openpyxl")
                cols = [str(c).strip() for c in df.columns]
                df.columns = cols
                if "Año" in cols and "Trimestre" in cols and any(c in cols for c in CITIES):
                    return df.assign(__sheet__=sh, __header_row__=hdr)
            except Exception:
                pass
    return pd.DataFrame()

def annualize(df_long, method="Q4"):
    q_order = {"I":1, "II":2, "III":3, "IV":4}
    df = df_long.copy()
    df["q"] = df["Trimestre"].map(q_order)
    if method.upper() == "MEAN":
        out = (df.groupby(["ciudad","Año"], as_index=False)
                 .agg(ipvn_var_anual_pct=("ipvn_var_anual_pct","mean")))
        return out
    # método Q4 por defecto
    def pick_row(g):
        g = g.dropna(subset=["ipvn_var_anual_pct"]).sort_values("q")
        sel = g[g["Trimestre"]=="IV"]
        if sel.empty:
            sel = g.tail(1)
        return sel.iloc[[0]][["ciudad","Año","ipvn_var_anual_pct"]]
    parts = []
    for (_, _), g in df.groupby(["ciudad","Año"]):
        if g.dropna(subset=["ipvn_var_anual_pct"]).empty:
            continue
        parts.append(pick_row(g))
    return pd.concat(parts, ignore_index=True) if parts else pd.DataFrame(columns=["ciudad","Año","ipvn_var_anual_pct"])

# 3) Leer Excel
assert os.path.exists(IPVN_LOCAL_FILE), f"No se encuentra el archivo: {IPVN_LOCAL_FILE}"
xls = pd.ExcelFile(IPVN_LOCAL_FILE, engine="openpyxl")
ipvn_table = try_read_target_sheet(xls)
print("Hoja encontrada:", ipvn_table["__sheet__"].iat[0], "| header row:", ipvn_table["__header_row__"].iat[0])
print("Forma Inicial:", ipvn_table.shape)
display(ipvn_table.head(10))

# 4) Selección de columnas y formato
cols_needed = ["Año","Trimestre"] + [c for c in CITIES if c in ipvn_table.columns]
missing = set(["Año","Trimestre"]) - set(ipvn_table.columns)
assert not missing, f"Faltan columnas clave en el archivo: {missing}"

wide = ipvn_table[cols_needed].copy()

# convercion de ciudades
long = wide.melt(id_vars=["Año","Trimestre"],
                 value_vars=[c for c in CITIES if c in wide.columns],
                 var_name="ciudad", value_name="ipvn_var_anual_pct")

# Limpieza
long["Año"] = pd.to_numeric(long["Año"], errors="coerce")
long["ipvn_var_anual_pct"] = pd.to_numeric(long["ipvn_var_anual_pct"], errors="coerce")
long = long.dropna(subset=["Año","ipvn_var_anual_pct"])
print("\n Formato (trimestral) — shape:", long.shape)
display(long.head(12))

# 5) Filtro 2020–2024 y anualización
long_2020_24 = long[(long["Año"]>=YEAR_START) & (long["Año"]<=YEAR_END)].copy()
ipvn_annual = annualize(long_2020_24, method=AGG_METHOD)

# Orden y renombre de columnas
ipvn_annual = (ipvn_annual
               .rename(columns={"Año":"year"})
               .sort_values(["ciudad","year"])
               .reset_index(drop=True))

print(f"\n IPVN anual por ciudad ({AGG_METHOD})  {YEAR_START}-{YEAR_END} — shape:", ipvn_annual.shape)
display(ipvn_annual.head(12))

# 6) Guardado
ipvn_long = ipvn_annual.rename(columns={"ipvn_var_anual_pct":"ipvn_valor"}).copy()

ipvn_annual.to_csv("ipvn_ciudades_2020_2024_anual_Q4.csv", index=False)
ipvn_long.to_csv("ipvn_ciudades_2020_2024.csv", index=False)   # <- 'ipvn_valor' para compatibilidad

print("Listo: ipvn_ciudades_2020_2024_anual_Q4.csv y ipvn_ciudades_2020_2024.csv")

Hoja encontrada: Anual | header row: 0
Forma Inicial: (110, 9)


,Año,Trimestre,Total,Barranquilla,Bogotá,Cali,Medellín,__sheet__,__header_row__
0,1998,I,13.27,40.49,14.49,4.96,9.18,Anual,0
1,1998,II,11.68,30.06,11.14,6.32,10.57,Anual,0
2,1998,III,8.25,22.19,7.74,2.16,5.79,Anual,0
3,1998,IV,5.37,11.91,3.66,0.26,7.38,Anual,0
4,1999,I,3.29,10.86,1.97,0.90,3.40,Anual,0
5,1999,II,1.93,8.41,1.26,-0.87,2.22,Anual,0
6,1999,III,1.79,9.49,1.39,-1.08,2.88,Anual,0
7,1999,IV,-0.49,6.68,0.03,4.00,-2.27,Anual,0
8,2000,I,-0.18,2.84,0.50,3.34,-0.90,Anual,0
9,2000,II,-3.22,1.11,-3.57,1.85,-2.58,Anual,0



 Formato (trimestral) — shape: (440, 4)


,Año,Trimestre,ciudad,ipvn_var_anual_pct
0,1998,I,Bogotá,14.49
1,1998,II,Bogotá,11.14
2,1998,III,Bogotá,7.74
3,1998,IV,Bogotá,3.66
4,1999,I,Bogotá,1.97
5,1999,II,Bogotá,1.26
6,1999,III,Bogotá,1.39
7,1999,IV,Bogotá,0.03
8,2000,I,Bogotá,0.50
9,2000,II,Bogotá,-3.57



 IPVN anual por ciudad (Q4)  2020-2024 — shape: (20, 3)


,ciudad,year,ipvn_var_anual_pct
0,Barranquilla,2020,3.21
1,Barranquilla,2021,6.57
2,Barranquilla,2022,7.54
3,Barranquilla,2023,14.23
4,Barranquilla,2024,14.77
5,Bogotá,2020,2.14
6,Bogotá,2021,8.66
7,Bogotá,2022,9.05
8,Bogotá,2023,6.95
9,Bogotá,2024,5.93


Listo: ipvn_ciudades_2020_2024_anual_Q4.csv y ipvn_ciudades_2020_2024.csv


In [11]:
# ============================================
# 3.1) Visualizacion IPVN con Plotly
# ============================================
import pandas as pd
import plotly.express as px

# Copia y orden básico
_ipvn = ipvn_long.copy().sort_values(["ciudad", "year"])

# (Opcional) Vista rápida
print("ipvn_long shape:", _ipvn.shape)
display(_ipvn.head(8))

# 1) Línea: variación anual por ciudad (2020–2024)
fig_ipvn_line = px.line(
    _ipvn, x="year", y="ipvn_valor", color="ciudad", markers=True,
    labels={"year":"Año", "ipvn_valor":"Variación anual IPVN (%)"},
    title="IPVN — Variación anual por ciudad"
)
fig_ipvn_line.update_layout(
    hovermode="x unified",
    xaxis=dict(dtick=1),
    yaxis_ticksuffix="%",
    legend_title_text="Ciudad",
    template="plotly_white"
)
fig_ipvn_line.show()

# 2) Barras agrupadas: comparación por año y ciudad
fig_ipvn_bar = px.bar(
    _ipvn, x="year", y="ipvn_valor", color="ciudad", barmode="group",
    text="ipvn_valor",
    labels={"year":"Año", "ipvn_valor":"Variación anual IPVN (%)"},
    title="IPVN — Variación anual por ciudad (barras agrupadas)"
)
fig_ipvn_bar.update_traces(texttemplate="%{text:.1f}%", textposition="outside")
fig_ipvn_bar.update_layout(
    xaxis=dict(dtick=1),
    yaxis_ticksuffix="%",
    uniformtext_minsize=8, uniformtext_mode="hide",
    legend_title_text="Ciudad",
    template="plotly_white"
)
fig_ipvn_bar.show()

# Guardar gráficas SMMLV en HTML (carpeta figs_SMLV)
from pathlib import Path

# Crear carpeta figs_SMLV en la misma ruta del proyecto
out_dir = Path("figs_IPVN")
out_dir.mkdir(parents=True, exist_ok=True)

# Rango de años para nombres
y0, y1 = int(_ipvn["year"].min()), int(_ipvn["year"].max())

# Guardar HTML interactivo
fig1.write_html(str(out_dir / f"smmlv_barras_{y0}_{y1}.html"), include_plotlyjs="cdn")
fig2.write_html(str(out_dir / f"smmlv_variacion_anual_{y0}_{y1}.html"), include_plotlyjs="cdn")



ipvn_long shape: (20, 3)


,ciudad,year,ipvn_valor
0,Barranquilla,2020,3.21
1,Barranquilla,2021,6.57
2,Barranquilla,2022,7.54
3,Barranquilla,2023,14.23
4,Barranquilla,2024,14.77
5,Bogotá,2020,2.14
6,Bogotá,2021,8.66
7,Bogotá,2022,9.05


In [12]:
# ============================================
# 4) Visualizaciones SMMLV con periodo 2020-2024 (Plotly)
# ============================================
import os
import pandas as pd
import plotly.express as px

# 1) Obtener SMMLV
if "smmlv" not in globals() or smmlv is None or getattr(smmlv, "empty", True):
    if os.path.exists("smmlv_2020_2024.csv"):
        smmlv = pd.read_csv("smmlv_2020_2024.csv")
    else:
        smmlv = pd.DataFrame({
            "year": [2020, 2021, 2022, 2023, 2024],
            "SMMLV": [877803, 908526, 1_000_000, 1_160_000, 1_300_000]
        })
        smmlv.to_csv("smmlv_2020_2024.csv", index=False)

smmlv["year"] = pd.to_numeric(smmlv["year"], errors="coerce")
smmlv["SMMLV"] = pd.to_numeric(smmlv["SMMLV"], errors="coerce")

# 2) Cálculo y plots
_sm = smmlv.copy()
_sm["var_anual_pct"] = _sm["SMMLV"].pct_change() * 100

# 3) Barra SMMLV
fig1 = px.bar(
    _sm, x="year", y="SMMLV", text="SMMLV",
    labels={"year":"Año", "SMMLV":"SMMLV (COP/mes)"},
    title="SMMLV 2020–2024 (COP/mes)"
)
fig1.update_traces(texttemplate="%{text:,.0f}", textposition="outside", marker_color="#1f77b4")
fig1.update_layout(yaxis_tickformat=",.0f", uniformtext_minsize=8, uniformtext_mode="hide", template="plotly_white")
fig1.show()

# 4) Línea variación anual
fig2 = px.line(
    _sm, x="year", y="var_anual_pct", markers=True,
    labels={"year":"Año", "var_anual_pct":"Variación anual (%)"},
    title="Variación anual del SMMLV"
)
fig2.update_traces(line_color="#d62728")
fig2.update_layout(yaxis_ticksuffix="%", hovermode="x unified", template="plotly_white")
fig2.show()

#Generacion de imagen html

try:
    base_dir = Path(BANREP_CSV).parent
except NameError:
    base_dir = Path.cwd()

out_dir = base_dir / "figs_SMLV"
out_dir.mkdir(parents=True, exist_ok=True)


# 3) Barra SMMLV (COP/mes)
fig1 = px.bar(
    _sm, x="year", y="SMMLV", text="SMMLV",
    labels={"year":"Año", "SMMLV":"SMMLV (COP/mes)"},
    title="SMMLV 2020–2024 (COP/mes)"
)
fig1.update_traces(texttemplate="%{text:,.0f}", textposition="outside", marker_color="#1f77b4")
fig1.update_layout(yaxis_tickformat=",.0f", uniformtext_minsize=8, uniformtext_mode="hide", template="plotly_white")
fig1.show()

# 4) Línea variación anual (%)
fig2 = px.line(
    _sm, x="year", y="var_anual_pct", markers=True,
    labels={"year":"Año", "var_anual_pct":"Variación anual (%)"},
    title="Variación anual del SMMLV"
)
fig2.update_traces(line_color="#d62728")
fig2.update_layout(yaxis_ticksuffix="%", hovermode="x unified", template="plotly_white")
fig2.show()

# 5) Guardar solo en HTML (sin Kaleido) en figs_SMLV
y0, y1 = int(_sm["year"].min()), int(_sm["year"].max())
fname_bar_html = out_dir / f"smmlv_barras_{y0}_{y1}.html"
fname_line_html = out_dir / f"smmlv_variacion_anual_{y0}_{y1}.html"

fig1.write_html(str(fname_bar_html),  include_plotlyjs="cdn", full_html=True)
fig2.write_html(str(fname_line_html), include_plotlyjs="cdn", full_html=True)




In [13]:
# ============================================
# 5) Indicadores - KPIs, basados en las tasa del BanRep
# Como las tasas del BanRep no estaban por municipio, se generaliza a nivel nacional
# basado tambien en el IPVN
# ============================================

import pandas as pd
import numpy as np

PLAZO_YEARS = globals().get("PLAZO_YEARS", 20)
N_MESES     = PLAZO_YEARS * 12
PRINCIPAL   = 100_000_000  # COP

nat = pd.read_csv("sfc_tasas_mensuales_nacional.csv", parse_dates=["mes"])
nat["year"] = nat["mes"].dt.year

tasas_nat_year = (nat.groupby("year", as_index=False)
                    .agg(tasa_mensual_prom=("tasa_mensual","mean"),
                         tasa_ea_prom=("tasa_media_ea","mean"),
                         n_meses=("tasa_mensual","size")))

# SMMLV
if "smmlv" not in globals() or smmlv is None or getattr(smmlv, "empty", True):
    smmlv = pd.read_csv("smmlv_2020_2024.csv")

smmlv["year"] = pd.to_numeric(smmlv["year"], errors="coerce")
smmlv["SMMLV"] = pd.to_numeric(smmlv["SMMLV"], errors="coerce")

indic_nat = tasas_nat_year.merge(smmlv, on="year", how="left")

# Cuota por $100M y cuota/SMMLV (nacional)
i = indic_nat["tasa_mensual_prom"].fillna(0.0)
n = N_MESES
indic_nat["cuota_por_100m"] = np.where(
    i > 0, PRINCIPAL * (i * (1+i)**n) / ((1+i)**n - 1), PRINCIPAL / n
)
indic_nat["cuota_en_smmlv"] = indic_nat["cuota_por_100m"] / indic_nat["SMMLV"]

print("📊 Indicadores nacionales:")
display(indic_nat[["year","tasa_ea_prom","tasa_mensual_prom","n_meses","SMMLV","cuota_por_100m","cuota_en_smmlv"]]
        .style.format({"tasa_ea_prom":"{:.2%}","tasa_mensual_prom":"{:.2%}","cuota_por_100m":"{:,.0f}","SMMLV":"{:,.0f}","cuota_en_smmlv":"{:.2f}"}))



📊 Indicadores nacionales:


,year,tasa_ea_prom,tasa_mensual_prom,n_meses,SMMLV,cuota_por_100m,cuota_en_smmlv
0,2020,11.55%,0.92%,12,"877,803","1,030,941",1.17
1,2021,10.70%,0.85%,12,"908,526","978,916",1.08
2,2022,13.01%,1.02%,12,"1,000,000","1,120,878",1.12
3,2023,15.21%,1.19%,12,"1,160,000","1,260,872",1.09
4,2024,12.67%,1.00%,12,"1,300,000","1,099,740",0.85


In [14]:
#============================================
# 5.1) Indicadores de Asequibilidad
# ============================================
import os
import numpy as np
import pandas as pd
from IPython.display import display

# Parámetros del modelo de crédito
PLAZO_YEARS = 20
N_MESES     = PLAZO_YEARS * 12
PRINCIPAL   = 100_000_000  # COP

# 5.1 Insumos
assert os.path.exists("sfc_tasas_mensuales_nacional.csv"), "Falta archivo con tasas."
nat = pd.read_csv("sfc_tasas_mensuales_nacional.csv", parse_dates=["mes"])

if "smmlv" not in globals() or smmlv is None or getattr(smmlv, "empty", True):
    assert os.path.exists("smmlv_2020_2024.csv"), "Falta smmlv_2020_2024.csv."
    smmlv = pd.read_csv("smmlv_2020_2024.csv")

for c in ["tasa_media_ea","tasa_mensual","n_obs"]:
    if c in nat.columns:
        nat[c] = pd.to_numeric(nat[c], errors="coerce")

nat["year"] = nat["mes"].dt.year
years_present = sorted(nat["year"].unique().tolist())

# 5.2 Indicadores nacionales (por año)
tasas_nat_year = (nat.groupby("year", as_index=False)
                    .agg(tasa_mensual_prom=("tasa_mensual","mean"),
                         tasa_ea_prom=("tasa_media_ea","mean"),
                         n_meses=("tasa_mensual","size")))

smmlv["year"]  = pd.to_numeric(smmlv["year"], errors="coerce")
smmlv["SMMLV"] = pd.to_numeric(smmlv["SMMLV"], errors="coerce")

indicadores_nacionales = tasas_nat_year.merge(smmlv, on="year", how="left")

i = indicadores_nacionales["tasa_mensual_prom"].fillna(0.0)
n = N_MESES
indicadores_nacionales["cuota_por_100m"] = np.where(
    i > 0, PRINCIPAL * (i * (1+i)**n) / ((1+i)**n - 1), PRINCIPAL / n
)
indicadores_nacionales["cuota_en_smmlv"] = indicadores_nacionales["cuota_por_100m"] / indicadores_nacionales["SMMLV"]

# Mostrar
print(" Indicadores nacionales")
display(indicadores_nacionales.style.format({
    "tasa_ea_prom":"{:.2%}", "tasa_mensual_prom":"{:.2%}",
    "SMMLV":"{:,.0f}", "cuota_por_100m":"{:,.0f}", "cuota_en_smmlv":"{:.2f}"
}))

# Guardar archivo de indicador
indicadores_nacionales.to_csv("indicadores_nacionales.csv", index=False)
print("Guardado: indicadores_nacionales.csv")

# 5.3 Indicadores por ciudad (va a ser igual por las tasa de BanRep no esta por municipio)
if os.path.exists("sfc_tasas_mensuales_ciudad.csv"):
    sfc_city = pd.read_csv("sfc_tasas_mensuales_ciudad.csv", parse_dates=["mes"])
    for c in ["tasa_media_ea","tasa_mensual","n_obs"]:
        if c in sfc_city.columns:
            sfc_city[c] = pd.to_numeric(sfc_city[c], errors="coerce")
    sfc_city["year"] = sfc_city["mes"].dt.year

    tasas_city_year = (sfc_city.groupby(["ciudad","year"], as_index=False)
                                .agg(tasa_mensual_prom=("tasa_mensual","mean"),
                                     tasa_ea_prom=("tasa_media_ea","mean"),
                                     n_meses=("tasa_mensual","size")))

    indicadores_ciudad = tasas_city_year.merge(smmlv, on="year", how="left")
    i_c = indicadores_ciudad["tasa_mensual_prom"].fillna(0.0)
    indicadores_ciudad["cuota_por_100m"] = np.where(
        i_c > 0, PRINCIPAL * (i_c * (1+i_c)**n) / ((1+i_c)**n - 1), PRINCIPAL / n
    )
    indicadores_ciudad["cuota_en_smmlv"] = indicadores_ciudad["cuota_por_100m"] / indicadores_ciudad["SMMLV"]

    print("Indicadores por ciudad (nota: con BanRep serán iguales entre ciudades)")
    display(indicadores_ciudad.head(8).style.format({
        "tasa_ea_prom":"{:.2%}", "tasa_mensual_prom":"{:.2%}",
        "SMMLV":"{:,.0f}", "cuota_por_100m":"{:,.0f}", "cuota_en_smmlv":"{:.2f}"
    }))

    indicadores_ciudad.to_csv("indicadores_por_ciudad.csv", index=False)



 Indicadores nacionales


,year,tasa_mensual_prom,tasa_ea_prom,n_meses,SMMLV,cuota_por_100m,cuota_en_smmlv
0,2020,0.92%,11.55%,12,"877,803","1,030,941",1.17
1,2021,0.85%,10.70%,12,"908,526","978,916",1.08
2,2022,1.02%,13.01%,12,"1,000,000","1,120,878",1.12
3,2023,1.19%,15.21%,12,"1,160,000","1,260,872",1.09
4,2024,1.00%,12.67%,12,"1,300,000","1,099,740",0.85


Guardado: indicadores_nacionales.csv
Indicadores por ciudad (nota: con BanRep serán iguales entre ciudades)


,ciudad,year,tasa_mensual_prom,tasa_ea_prom,n_meses,SMMLV,cuota_por_100m,cuota_en_smmlv
0,Barranquilla,2020,0.92%,11.55%,12,"877,803","1,030,941",1.17
1,Barranquilla,2021,0.85%,10.70%,12,"908,526","978,916",1.08
2,Barranquilla,2022,1.02%,13.01%,12,"1,000,000","1,120,878",1.12
3,Barranquilla,2023,1.19%,15.21%,12,"1,160,000","1,260,872",1.09
4,Barranquilla,2024,1.00%,12.67%,12,"1,300,000","1,099,740",0.85
5,Bogotá,2020,0.92%,11.55%,12,"877,803","1,030,941",1.17
6,Bogotá,2021,0.85%,10.70%,12,"908,526","978,916",1.08
7,Bogotá,2022,1.02%,13.01%,12,"1,000,000","1,120,878",1.12


In [15]:
# ============================================
# 6) Visualizaciones indicadores nacionales
# (se generaliza a nivel nacional por no contar con las tasas por municipio)
# ============================================
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

assert os.path.exists("sfc_tasas_mensuales_nacional.csv"), "Falta sfc_tasas_mensuales_nacional.csv"
nat = pd.read_csv("sfc_tasas_mensuales_nacional.csv", parse_dates=["mes"])
for c in ["tasa_media_ea","tasa_mensual","n_obs"]:
    if c in nat.columns:
        nat[c] = pd.to_numeric(nat[c], errors="coerce")

nat = nat.sort_values("mes").copy()
nat["year"] = nat["mes"].dt.year
nat["month"] = nat["mes"].dt.month
nat["mm3"] = nat["tasa_mensual"].rolling(3, min_periods=1).mean()

years = nat["year"].unique()
y0, y1 = int(years.min()), int(years.max())

os.makedirs("vis_banrep", exist_ok=True)

# 6.1 Línea mensual + Media Movil (MM3) + picos/vales
i_max = nat["tasa_mensual"].idxmax()
i_min = nat["tasa_mensual"].idxmin()

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=nat["mes"], y=nat["tasa_mensual"],
                          mode="lines+markers", name="Tasa mensual",
                          line=dict(color="#1f77b4")))
fig1.add_trace(go.Scatter(x=nat["mes"], y=nat["mm3"],
                          mode="lines", name="Media móvil 3M",
                          line=dict(color="#ff7f0e", dash="dash")))
if not np.isnan(i_max):
    fig1.add_trace(go.Scatter(
        x=[nat.loc[i_max,"mes"]], y=[nat.loc[i_max,"tasa_mensual"]],
        mode="markers+text", text=["Máximo"], textposition="top center",
        name="Máximo", marker=dict(color="#d62728", size=10)))
if not np.isnan(i_min):
    fig1.add_trace(go.Scatter(
        x=[nat.loc[i_min,"mes"]], y=[nat.loc[i_min,"tasa_mensual"]],
        mode="markers+text", text=["Mínimo"], textposition="bottom center",
        name="Mínimo", marker=dict(color="#2ca02c", size=10)))

fig1.update_layout(
    title=f"Tasa hipotecaria promedio — nacional (mensual) {y0}–{y1}",
    xaxis_title="Mes", yaxis_title="Tasa mensual",
    template="plotly_white", hovermode="x unified")
fig1.update_yaxes(tickformat=".2%")
fig1.update_xaxes(rangeslider_visible=True)
fig1.write_html("vis_banrep/fig_tasa_mensual_linea.html")
fig1.show()

# 6.2 Barras anuales (EA) con cobertura
nat_y = (nat.groupby("year", as_index=False)
            .agg(tasa_ea_prom=("tasa_media_ea","mean"),
                 n_meses=("tasa_mensual","size")))

fig2 = px.bar(nat_y, x="year", y="tasa_ea_prom",
              text=nat_y["tasa_ea_prom"].map(lambda v: f"{v:.2%}"),
              labels={"year":"Año","tasa_ea_prom":"Tasa EA promedio (nacional)"},
              title="Tasa EA promedio por año — Nacional")
fig2.update_traces(textposition="outside", marker_color="#4e79a7")
fig2.update_layout(template="plotly_white", xaxis=dict(dtick=1), yaxis_tickformat=".2%",
                   annotations=[
                       dict(x=int(r.year), y=float(r.tasa_ea_prom), xanchor="center", yanchor="bottom",
                            text=f"<b>n={int(r.n_meses)}</b>", showarrow=False, yshift=18, font=dict(color="#333"))
                       for r in nat_y.itertuples(index=False)
                   ])
fig2.write_html("vis_banrep/fig_tasa_ea_anual_barras.html")
fig2.show()

# 6.3 Mapa de calor mensual (año/mes)
heat = (nat.pivot_table(index="year", columns="month", values="tasa_mensual", aggfunc="mean")
           .sort_index())
fig3 = px.imshow(heat, aspect="auto", color_continuous_scale="Viridis",
                 labels=dict(color="Tasa mensual"), title="Heatmap — Tasa mensual (nacional) por año/mes")
fig3.update_layout(template="plotly_white")
fig3.update_coloraxes(colorbar_title="Tasa")
fig3.update_yaxes(title="Año")
fig3.update_xaxes(title="Mes")
fig3.write_html("vis_banrep/fig_heatmap_mensual.html")
fig3.show()

# 6.4 Boxplot por año (distribución mensual) ---
fig4 = px.box(nat, x="year", y="tasa_mensual",
              labels={"year":"Año","tasa_mensual":"Tasa mensual"},
              title="Distribución de la tasa mensual por año (nacional)")
fig4.update_layout(template="plotly_white")
fig4.update_yaxes(tickformat=".2%")
fig4.write_html("vis_banrep/fig_boxplot_anual.html")
fig4.show()


In [16]:
# ============================================
# 7) Conclusiones a partir de las informacion del Banco de la Republica
# generacion de markdown
# ============================================
import os
import pandas as pd
import numpy as np
from IPython.display import display

assert os.path.exists("indicadores_nacionales.csv"), "Falta indicadores_nacionales.csv."
ind_nat = pd.read_csv("indicadores_nacionales.csv")

# Formatos / cálculos auxiliares
ind_nat["tasa_ea_prom"] = pd.to_numeric(ind_nat["tasa_ea_prom"], errors="coerce")
ind_nat["cuota_en_smmlv"] = pd.to_numeric(ind_nat["cuota_en_smmlv"], errors="coerce")
ind_nat["cuota_por_100m"] = pd.to_numeric(ind_nat["cuota_por_100m"], errors="coerce")
ind_nat["SMMLV"] = pd.to_numeric(ind_nat["SMMLV"], errors="coerce")

ymin, ymax = int(ind_nat["year"].min()), int(ind_nat["year"].max())
fila_ult = ind_nat[ind_nat["year"]==ymax].iloc[0]

# Resumen nacional
texto = []
texto.append(f"# Conclusiones — Asequibilidad (BanRep, {ymin}–{ymax})\n")
texto.append("**Fuente de tasas:** Banco de la República (tasa de colocación mensual en PESOS, EA).")
texto.append("- La tasa se transformó a **tasa mensual** con \( r_m = (1+r_{EA})^{1/12}-1 \).")
texto.append("- La tasa es **nacional**; por compatibilidad, se replica a 4 ciudades en las vistas por ciudad.\n")

texto.append("## Cifras clave")
texto.append(f"- **Promedio EA {ymin}–{ymax}:** {ind_nat['tasa_ea_prom'].mean():.2%}")
texto.append(f"- **{ymax}:** EA {fila_ult['tasa_ea_prom']:.2%} → **cuota** {fila_ult['cuota_por_100m']:,.0f} COP/mes por cada $100M, "
             f"equivalente a **{fila_ult['cuota_en_smmlv']:.2f} SMMLV**.\n")

# Tendencia (si hay >1 año)
if len(ind_nat) >= 2:
    fila_ini = ind_nat[ind_nat["year"]==ymin].iloc[0]
    delta_ea = fila_ult["tasa_ea_prom"] - fila_ini["tasa_ea_prom"]
    delta_cuota_smmlv = fila_ult["cuota_en_smmlv"] - fila_ini["cuota_en_smmlv"]
    texto.append("## Tendencia")
    texto.append(f"- De **{ymin}** a **{ymax}**: EA varió **{delta_ea:+.2%}**; "
                 f"la **cuota/SMMLV** cambió **{delta_cuota_smmlv:+.2f}**.\n")

#Reseña IPVN:
if os.path.exists("ipvn_ciudades_2020_2024.csv"):
    ipvn = pd.read_csv("ipvn_ciudades_2020_2024.csv")
    # estandarizar nombre si fuese distinto
    if "ipvn_var_anual_pct" in ipvn.columns:
        ipvn["ipvn_var_anual"] = pd.to_numeric(ipvn["ipvn_var_anual_pct"], errors="coerce")
    elif "ipvn_valor" in ipvn.columns:
        ipvn["ipvn_var_anual"] = pd.to_numeric(ipvn["ipvn_valor"], errors="coerce")
    else:
        ipvn["ipvn_var_anual"] = np.nan

    ipvn_y = ipvn[ipvn["year"]==ymax].dropna(subset=["ipvn_var_anual"])
    if not ipvn_y.empty:
        top = ipvn_y.sort_values("ipvn_var_anual", ascending=False).head(2)
        bot = ipvn_y.sort_values("ipvn_var_anual", ascending=True).head(2)
        texto.append("## IPVN (variación anual, por ciudad)")
        texto.append(f"- **{ymax}**: mayores aumentos de precios en: " +
                     "; ".join([f"{r.ciudad} ({r.ipvn_var_anual:.2f}%)" for r in top.itertuples(index=False)]))
        texto.append(f"- **{ymax}**: menores aumentos (o caídas) en: " +
                     "; ".join([f"{r.ciudad} ({r.ipvn_var_anual:.2f}%)" for r in bot.itertuples(index=False)]) + "\n")

# Guardar markdown
md = "\n".join(texto)
with open("conclusiones_resumen.md", "w", encoding="utf-8") as f:
    f.write(md)
print("Guardado: conclusiones_resumen.md")
print("\n Vista previa:")
print(md)

# Conclusiones por ciudad (si generaste indicadores_por_ciudad.csv)
if os.path.exists("indicadores_por_ciudad.csv"):
    by_city = pd.read_csv("indicadores_por_ciudad.csv")
    print("\n Conclusiones por ciudad (nota: BanRep → cifras iguales entre ciudades)")
    display(by_city.head(8).style.format({
        "tasa_ea_prom":"{:.2%}", "tasa_mensual_prom":"{:.2%}",
        "cuota_por_100m":"{:,.0f}", "SMMLV":"{:,.0f}", "cuota_en_smmlv":"{:.2f}"
    }))
    by_city.to_csv("conclusiones_por_ciudad.csv", index=False)


Guardado: conclusiones_resumen.md

 Vista previa:
# Conclusiones — Asequibilidad (BanRep, 2020–2024)

**Fuente de tasas:** Banco de la República (tasa de colocación mensual en PESOS, EA).
- La tasa se transformó a **tasa mensual** con \( r_m = (1+r_{EA})^{1/12}-1 \).
- La tasa es **nacional**; por compatibilidad, se replica a 4 ciudades en las vistas por ciudad.

## Cifras clave
- **Promedio EA 2020–2024:** 12.63%
- **2024:** EA 12.67% → **cuota** 1,099,740 COP/mes por cada $100M, equivalente a **0.85 SMMLV**.

## Tendencia
- De **2020** a **2024**: EA varió **+1.11%**; la **cuota/SMMLV** cambió **-0.33**.

## IPVN (variación anual, por ciudad)
- **2024**: mayores aumentos de precios en: Barranquilla (14.77%); Medellín (10.42%)
- **2024**: menores aumentos (o caídas) en: Bogotá (5.93%); Cali (7.75%)


 Conclusiones por ciudad (nota: BanRep → cifras iguales entre ciudades)


,ciudad,year,tasa_mensual_prom,tasa_ea_prom,n_meses,SMMLV,cuota_por_100m,cuota_en_smmlv
0,Barranquilla,2020,0.92%,11.55%,12,"877,803","1,030,941",1.17
1,Barranquilla,2021,0.85%,10.70%,12,"908,526","978,916",1.08
2,Barranquilla,2022,1.02%,13.01%,12,"1,000,000","1,120,878",1.12
3,Barranquilla,2023,1.19%,15.21%,12,"1,160,000","1,260,872",1.09
4,Barranquilla,2024,1.00%,12.67%,12,"1,300,000","1,099,740",0.85
5,Bogotá,2020,0.92%,11.55%,12,"877,803","1,030,941",1.17
6,Bogotá,2021,0.85%,10.70%,12,"908,526","978,916",1.08
7,Bogotá,2022,1.02%,13.01%,12,"1,000,000","1,120,878",1.12


In [ ]:
# ============================================
# 8) Carga a Base datos SQL las tablas generadas
# para carga de tablas en BD, pero seria para uso local
# se tiene como motor de base de dtaos postgress y esta en forma local, Colab no permite la conexion,
# por lo tanto se podria ejecutar en VS para su conexio local.
# ============================================

!pip install --quiet SQLAlchemy==2.0.31 psycopg2-binary==2.9.9 pandas==2.2.2

import os
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.types import Date, DateTime, Integer, Float, Numeric, Text

# --- Config conn ---
PG_USER = "arq"
PG_PASS = "password"
PG_HOST = "localhost"
PG_PORT = 5432
PG_DB   = "bd"

PG_URL  = f"postgresql+psycopg2://{PG_USER}:{PG_PASS}@{PG_HOST}:{PG_PORT}/{PG_DB}"
engine  = create_engine(PG_URL, pool_pre_ping=True)

#---------------------------------------------------------------

from sqlalchemy.types import Date, Integer, Float, Numeric, Text

# 3.1 Catálogo de ciudades (fijo)
dim_ciudad = pd.DataFrame({
    "ciudad": ["Bogotá","Medellín","Cali","Barranquilla"],
    "codigo_municipio": ["11001","05001","76001","08001"]
})

# 3.2 Mapeos de tipos para columnas clave
dtype_sfc_nat = {
    "mes": Date(),
    "tasa_media_ea": Numeric(18,10),
    "tasa_mensual": Numeric(18,10),
    "n_obs": Float(),
    "tipo_tasa": Text()
}

dtype_sfc_city = {
    "ciudad": Text(),
    "mes": Date(),
    "tasa_media_ea": Numeric(18,10),
    "tasa_mensual": Numeric(18,10),
    "n_obs": Float(),
    "tipo_tasa": Text()
}

dtype_ind_nat = {
    "year": Integer(),
    "tasa_ea_prom": Numeric(18,10),
    "tasa_mensual_prom": Numeric(18,10),
    "n_meses": Integer(),
    "smmlv": Numeric(18,2),
    "cuota_por_100m": Numeric(18,2),
    "cuota_en_smmlv": Numeric(18,4)
}

dtype_ind_city = {
    "ciudad": Text(),
    "year": Integer(),
    "precio_objetivo": Numeric(18,2),
    "principal": Numeric(18,2),
    "tasa_ea_prom": Numeric(18,10),
    "tasa_mensual_prom": Numeric(18,10),
    "n_meses": Integer(),
    "smmlv": Numeric(18,2),
    "cuota_mensual": Numeric(18,2),
    "cuota_en_smmlv": Numeric(18,4)
}

dtype_ipvn = {"ciudad": Text(), "year": Integer(), "ipvn_var_anual": Numeric(18,4)}
dtype_smmlv = {"year": Integer(), "smmlv": Numeric(18,2)}

# 3.3 Cargar
with engine.begin() as conn:
    dim_ciudad.to_sql("dim_ciudad", conn, schema="vivienda", if_exists="replace", index=False,
                      dtype={"ciudad": Text(), "codigo_municipio": Text()})
    sfc_nat_month.to_sql("fact_sfc_tasas_mensual_nac", conn, schema="vivienda", if_exists="replace", index=False, dtype=dtype_sfc_nat)
    sfc_month.to_sql("fact_sfc_tasas_mensual_ciudad", conn, schema="vivienda", if_exists="replace", index=False, dtype=dtype_sfc_city)
    indic_nacionales.to_sql("fact_indicadores_nacionales", conn, schema="vivienda", if_exists="replace", index=False, dtype=dtype_ind_nat)
    indic_ciudad.to_sql("fact_indicadores_ciudad", conn, schema="vivienda", if_exists="replace", index=False, dtype=dtype_ind_city)
    ipvn[["ciudad","year","ipvn_var_anual"]].to_sql("dim_ipvn", conn, schema="vivienda", if_exists="replace", index=False, dtype=dtype_ipvn)
    smmlv[["year","smmlv"]].to_sql("dim_smmlv", conn, schema="vivienda", if_exists="replace", index=False, dtype=dtype_smmlv)

print("Tablas cargadas en PostgreSQL (esquema vivienda).")
